## Salient genes selection based on mutual information

In [ ]:
%%time
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
#import tensorflow as tf

#scikitlearn
from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mutual_info_score
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import scale
from sklearn.utils import check_random_state
from sklearn.utils.fixes import _astype_copy_false
from sklearn.utils.validation import check_array, check_X_y
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.utils.multiclass import check_classification_targets
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import svm
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from scipy import special, stats
from scipy.sparse import issparse
from sklearn.base import BaseEstimator
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import (as_float_array, check_array, check_X_y, safe_sqr,safe_mask)
from sklearn.utils.extmath import safe_sparse_dot, row_norms
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.feature_selection._base import SelectorMixin
from sklearn.metrics import roc_auc_score
#from sklearn.metrics.cluster import check_clusterings
from sklearn.metrics.cluster import contingency_matrix
from sklearn import metrics
from sklearn import preprocessing
from scipy.sparse import issparse
from scipy.special import digamma
import scipy.stats as stats

from subprocess import check_output

from math import log

import warnings

#keras
#import keras
#from keras.layers import Activation
#from tensorflow.keras import layers
#from keras.layers.normalization import BatchNormalization
#from keras import models
#from keras import layers
#from keras.layers import Input, Dense
#from keras.models import Model
#from keras import regularizers
#from keras.layers import Dropout
#from keras import backend as K

#lifelines
#from lifelines.datasets import load_waltons
#from lifelines import KaplanMeierFitter
#from lifelines.utils import median_survival_times
#from lifelines.datasets import load_regression_dataset
#from lifelines import CoxPHFitter

#garage collector
import gc

#networkx
#import networkx as nx

In [ ]:
#转置dataFrame
def trans(df1):
    df2=pd.DataFrame(df1.values.T,columns=df1.index,index=df1.columns)
    return df2 

In [ ]:
# Select the time-to-event time
def selected_time(selected_time, time, event):
    if (time > selected_time): return 0
    else:
        if (event == 1.0): return 1
        else: return None

In [ ]:
survival_metric='OS'
survival_time='OS.time'
time=1825
model_save_path='./saved_models/lung_model_cnn_{}_{}.h5'.format(survival_metric,time)

In [ ]:
%%time
Y_type=pd.read_hdf('data/Preprocessed_Data/PanCancer.h5',key='sample_type')
Y_clinical=pd.read_hdf('data/Preprocessed_Data/PanCancer.h5',key='clinical')

In [ ]:
Y_clinical=Y_clinical.loc[Y_type.pathology_status=='Tumor']
Y_clinical.dropna(subset=[survival_metric, survival_time], inplace=True)

In [ ]:
Y_clinical_categorical = Y_clinical[[survival_metric, survival_time]].apply(lambda row: selected_time(time, row[survival_time], row[survival_metric]), axis=1)
Y_clinical_categorical.dropna(inplace=True)
Y_clinical_categorical

In [ ]:
Y_type.head()

In [ ]:
Y_clinical.head()

In [ ]:
%%time
#读取基因表达数据
pan_gene_expre=pd.read_hdf('data/Preprocessed_Data/PanCancer.h5',key='gene_expression')

In [ ]:
X_gene_exp=pan_gene_expre.loc[[s for s in Y_clinical_categorical.index]]

In [ ]:
X_gene_exp

In [ ]:
%%time
pan_gene_expre.index.name='sample'
pan_gene_expre.columns.name='gene'

In [ ]:
pan_gene_expre

In [ ]:
X=X_gene_exp

In [ ]:
'''
%%time
pan_sample_types=pd.read_hdf('data/Preprocessed_Data/PanCancer.h5',key='sample_type')
pan_sample_types.head()
'''

In [ ]:
'''
%%time
y=pan_sample_types['sample_type_id']
y.head()
'''

In [ ]:
y=Y_clinical_categorical

In [ ]:
y.value_counts(normalize=True,ascending=False)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X=X.loc[[s for s in y.index]]

In [ ]:
%%time
#区分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
%%time
#varianceThreshold去掉在某一维完全一样以及某一维方差小于阈值的特征
#from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=15)
selector.fit(X_train) #只fit train才能保证train和test维度一样
X_train_filter = selector.transform(X_train)
X_test_filter = selector.transform(X_test)

In [ ]:
pd.DataFrame(X_train_filter)

In [ ]:
#尝试保留columns(基因）名
genes_name=X.columns.values.tolist()
select_name_index0=selector.get_support(indices=True)
select_name0=[]
for i in select_name_index0:
    select_name0.append(genes_name[i]) #这是剩下来的基因名
len(select_name0)

In [ ]:
%%time
X_train_T = X_train_filter.T
X_test_T = X_test_filter.T
X_train_T = pd.DataFrame(X_train_T)
X_test_T = pd.DataFrame(X_test_T)

In [ ]:
#还原基因名
X_train_T.index=select_name0
X_test_T.index=select_name0
print(X_train_T.head())
print(X_test_T.head())

In [ ]:
%%time
X_train_T.duplicated().sum()

In [ ]:
%%time
duplicated_features = X_train_T.duplicated()

In [ ]:
%%time
features_kept = [not index for index in duplicated_features]
X_train_unique = X_train_T[features_kept].T
X_test_unique = X_test_T[features_kept].T
X_train_unique.shape, X_test_unique.shape

In [ ]:
X_test_unique

In [ ]:
#用knn模型做二分类预测来评估特征选择
from sklearn.metrics import accuracy_score 
def test_knn(X_train, X_test, y_train, y_test):
    knn_model =KNeighborsClassifier()
    knn_model.fit(X_train, y_train)
    y_pred = knn_model.predict(X_train)
    y_test_pred=knn_model.predict(X_test)
    print('Training 80% data report::')
    print('Accuracy Score: ')
    print(accuracy_score(y_train, y_pred))
    print('F1-Score: ')
    print(f1_score(y_train, y_pred,average="weighted")) #考虑到类别的不平衡，故选择weighted
    print('Confusion Matrix: ')
    print(confusion_matrix(y_train, y_pred))
    print('Testing 20% data report::')
    print('Accuracy Score: ')
    print(accuracy_score(y_test, y_test_pred))
    print('F1-Score: ')
    print(f1_score(y_test, y_test_pred,average="weighted"))
    print('Confusion Matrix: ')
    print(confusion_matrix(y_test, y_test_pred))

In [ ]:
#用SVM~~~（速度极慢）
def test_svm(X_train, X_test, y_train, y_test):
    svm_model =svm.SVC()
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_train)
    y_test_pred=svm_model.predict(X_test)
    print('Training 80% data report::')
    print('Accuracy Score: ')
    print(accuracy_score(y_train, y_pred))
    print('F1-Score: ')
    print(f1_score(y_train, y_pred,average="weighted"))
    print('Confusion Matrix: ')
    print(confusion_matrix(y_train, y_pred))
    print('Testing 20% data report::')
    print('Accuracy Score: ')
    print(accuracy_score(y_test, y_test_pred))
    print('F1-Score: ')
    print(f1_score(y_test, y_test_pred,average="weighted"))
    print('Confusion Matrix: ')
    print(confusion_matrix(y_test, y_test_pred))

In [ ]:
'''
#driver code
def _compute_mi(x, y, n_neighbors=3):
        return mutual_info_score(x, y)

def _iterate_columns(X, columns=None):
    if columns is None:
        columns = range(X.shape[1])

    if issparse(X):
        for i in columns:
            x = np.zeros(X.shape[0])
            start_ptr, end_ptr = X.indptr[i], X.indptr[i + 1]
            x[X.indices[start_ptr:end_ptr]] = X.data[start_ptr:end_ptr]
            yield x
    else:
        for i in columns:
            yield X[:, i]


def _estimate_mi(X, y,n_neighbors=3):
    X, y = check_X_y(X, y, accept_sparse='csc')
    n_samples, n_features = X.shape
    discrete_features = issparse(X)
    discrete_mask = np.empty(n_features, dtype=bool)
    discrete_mask.fill(discrete_features)
    
    mi = [_compute_mi(x, y, n_neighbors) for
          x, discrete_feature in zip(_iterate_columns(X), discrete_mask)]

    return np.array(mi)

def mutual_info_classifier(X, y, n_neighbors=3,):
    check_classification_targets(y)
    return _estimate_mi(X, y, n_neighbors)
'''

In [ ]:
'''
%%time
mi = mutual_info_classifier(X_train_unique, y_train) 
len(mi)
'''

In [ ]:
'''
mi[: 10]
'''

In [ ]:
'''
%%time
mi = pd.Series(mi)
mi.index = X_train_unique.columns
'''

In [ ]:
'''
mi.sort_values(ascending=False, inplace = True)
'''

In [ ]:
%%time
#计算mutual information score
def mi_based_selector(X_train, y_train, X_test):
    fs = SelectKBest(score_func=mutual_info_classif, k=1000)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

In [ ]:
%%time
#选择特征
X_train_fs, X_test_fs, fs = mi_based_selector(X_train_unique, y_train, X_test_unique)

In [ ]:
#还原基因名
select_name_index=fs.get_support(indices=True)
select_name=[]
for i in select_name_index:
    select_name.append(select_name0[i])
#print(select_name)

In [ ]:
X_train_fs=pd.DataFrame(X_train_fs)
X_test_fs=pd.DataFrame(X_test_fs)
X_train_fs.columns=select_name
X_test_fs.columns=select_name
print(X_train_fs.head())
print(X_test_fs.head())

In [ ]:
'''
%%time
from sklearn.feature_selection import SelectKBest, SelectPercentile
sel = SelectKBest(mutual_info_classifier, k=20000).fit(X_train_unique, y_train)
X_train_unique.columns[sel.get_support()]
'''

In [ ]:
#len(X_train_unique.columns[sel.get_support()])

In [ ]:
'''
%%time
X_train_mi = sel.transform(X_train_unique)
X_test_mi = sel.transform(X_test_unique)
X_train_mi.shape
'''

In [ ]:
%%time
#用knn测试性能
test_knn(X_train_fs, X_test_fs, y_train, y_test)

In [ ]:
%%time
#测试未进行特征选择的性能
test_knn(X_train, X_test, y_train, y_test)

## Saving features

In [ ]:
pan_gene_expre[select_name]

In [ ]:
%%time
#保存筛选完的基因表达数据
with pd.HDFStore('data/Preprocessed_Data/pan_gene_filtered_11k.h5','w') as store:
    store['pan_gene_filtered']=pan_gene_expre[select_name]

## 接下来创建一个这20000个基因的肺癌样本表达数据文件

In [ ]:
%%time
#读取肺癌基因表达数据
lung_gene_expre=pd.read_hdf('data/Preprocessed_Data/Lung.h5',key='gene_expression')

In [ ]:
lung_gene_expre

In [ ]:
%%time
#保存筛选完的基因表达数据
with pd.HDFStore('data/Preprocessed_Data/lung_gene_filtered_10k.h5','w') as store:
    store['lung_gene_filtered']=lung_gene_expre[select_name]

In [ ]:
lung_gene_expre[select_name]